# Import thư viện

In [ ]:
# Nguyen Truong Huy.
import pyspark
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import col, count, when, to_timestamp, split, regexp_replace, row_number, sum, count_distinct
from functools import reduce
import pyspark.pandas as ps

# Khởi tạo Spark Session

In [ ]:
spark = SparkSession.builder \
    .appName("Preprocess") \
    .getOrCreate()

# Đọc file data

In [ ]:
raw_df = spark.read.csv("./data/raw_data.csv", header=True, inferSchema=True)

In [ ]:
raw_df.show()

In [ ]:
raw_df.describe().show()

In [ ]:
def dataframe_info(df):
    print(f"{'-'*40}")
    print(f"DataFrame thông tin:")
    print(f"Số dòng: {df.count()}")
    print(f"Số cột: {len(df.columns)}")
    print(f"{'-'*40}")
    print("Schema:")
    df.printSchema()
    print(f"{'-'*40}")
    print("Số giá trị null trong mỗi cột:")
    null_counts = df.select([
        count(when(col(c).isNull(), c)).alias(c) for c in df.columns
    ])
    null_counts.show()

In [ ]:
dataframe_info(raw_df)

# Tiền xử lí dữ liệu

### Xóa các cột không cần thiết

In [ ]:
preprocessed_data = raw_df.drop(*['thumbnail_link', 'comments_disabled', 'video_error_or_removed', 'ratings_disabled'])
dataframe_info(preprocessed_data)

### Xóa các hàng có tất cả các giá trị là Null

In [ ]:
preprocessed_data = preprocessed_data.filter(
    reduce(lambda a, b: a | b, (col(c).isNotNull() for c in preprocessed_data.columns))
)
preprocessed_data.show()

In [ ]:
dataframe_info(preprocessed_data)

### Xóa các hàng có trending_date sai định dạng (lỗi data => các giá trị khác trong hàng có nhiều giá trị Null)

In [ ]:
preprocessed_data = preprocessed_data.filter(
    col("trending_date").rlike(r"^\d{2}\.\d{2}\.\d{2}$")
)
dataframe_info(preprocessed_data)

### Thêm giá trị cho các hàng có cột description có giá trị bằng Null.

In [ ]:
preprocessed_data = preprocessed_data.fillna({"description": "No description"})
dataframe_info(preprocessed_data)

### Chuẩn hóa dữ liệu

In [ ]:
preprocessed_data = preprocessed_data.withColumn('trending_date', to_timestamp('trending_date', 'yy.dd.MM'))
preprocessed_data = preprocessed_data.withColumn('publish_time', to_timestamp('publish_time', "yyyy-MM-dd'T'HH:mm:ss.SSS'Z'"))

# Dataset rieng cho machine learning.
ML_data = preprocessed_data
ML_data = ML_data.withColumn('tags', when(preprocessed_data['tags'] == '[none]', '').otherwise(preprocessed_data['tags']))
ML_data = ML_data.withColumn('tags', split(regexp_replace("tags", '"', ""), "\\|"))

preprocessed_data.show()

In [ ]:
dataframe_info(preprocessed_data)

# Lưu dữ liệu đã được xử lý

In [ ]:
preprocessed_data.toPandas().to_csv('./data/preprocessed_data.csv')